# Movies - Part 1

According to the data dictionary, null values have been encoding as \N.
- You will want to find those and replace them with np.nan.
- However, the backslash (\) character is a special one that tells the computer to ignore whatever character comes next.
    - So if we were to say df.replace({'\N':np.nan}), the computer would see \N as an empty string.
    - To fix this, add a second backslash character, which will tell the computer that you actually WANTED to use a literal \.
    - df.replace({'\\N':np.nan}) 
    - Don't forget to make these replacements permanent!

In [5]:
import pandas as pd
import numpy as np

In [6]:
akas_url = 'https://datasets.imdbws.com/title.akas.tsv.gz'
basics_url = 'https://datasets.imdbws.com/title.basics.tsv.gz'
ratings_url = 'https://datasets.imdbws.com/title.ratings.tsv.gz'

### akas

AKAs:
- keep only us entries
- Replace '\N' with np.nan

In [7]:
akas = pd.read_csv(akas_url, sep='\t', low_memory=False)
akas.head(2)

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0


In [8]:
akas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31787691 entries, 0 to 31787690
Data columns (total 8 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   titleId          object
 1   ordering         int64 
 2   title            object
 3   region           object
 4   language         object
 5   types            object
 6   attributes       object
 7   isOriginalTitle  object
dtypes: int64(1), object(7)
memory usage: 1.9+ GB


In [9]:
akas = akas.replace({'\\N': np.nan})
akas = akas.dropna()
akas.isna().any()

titleId            False
ordering           False
title              False
region             False
language           False
types              False
attributes         False
isOriginalTitle    False
dtype: bool

In [10]:
akas = akas.loc[akas['region'] == 'US']

### Basics

Basics:
- Exclude any movie with missing values for genre or runtime
- Include only full-length movies (titleType = "movie").
- Include only fictional movies (not from documentary genre)
- Include only movies that were released 2000 - 2021 (include 2000 and 2021)
- Include only movies that were released in the United States

In [11]:
basics = pd.read_csv(basics_url, sep='\t', low_memory=False)
basics.head(2)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"


In [12]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8877960 entries, 0 to 8877959
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   primaryTitle    object
 3   originalTitle   object
 4   isAdult         object
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: object(9)
memory usage: 609.6+ MB


In [16]:
basics = basics.replace({'\\N': np.nan})
basics = basics.dropna(subset = ['runtimeMinutes', 'genres', 'startYear', 'endYear'])
basics.isna().any()

tconst            False
titleType         False
primaryTitle      False
originalTitle     False
isAdult           False
startYear         False
endYear           False
runtimeMinutes    False
genres            False
dtype: bool

In [17]:
basics = basics.loc[basics['titleType'] == 'movie']

In [18]:
basics['startYear'] = basics['startYear'].astype(int)
basics = basics.loc[(basics['startYear'] >= 2000) & (basics['startYear'] <= 2022)]

In [19]:
is_documentary = basics['genres'].str.contains('documentary', case=False)
basics = basics[~is_documentary]

### Ratings

Ratings:
- Replace "\N" with np.nan (if any)

In [20]:
ratings = pd.read_csv(ratings_url, sep='\t', low_memory=False)
ratings.head(2)

,tconst,averageRating,numVotes
0,tt0000001,5.7,1874
1,tt0000002,5.9,248


In [21]:
ratings.isna().any()

tconst           False
averageRating    False
numVotes         False
dtype: bool

## Filtering one dataframe based on another

In [22]:
# Filter the basics table down to only include the US by using the filter akas dataframe
keepers = basics['tconst'].isin(akas['titleId'])
# Now filter basics
basics = basics[keepers]
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres


In [23]:
# Making a new folder with os
import os
os.makedirs('Data/', exist_ok=True)
# Confirm folder created
os.listdir('Data/')

['title_akas.csv.gz', 'title_basics.csv.gz', 'title_ratings.csv.gz']

In [24]:
# Save current dataframe to file
akas.to_csv('Data/title_akas.csv.gz', compression='gzip', index=False)
basics.to_csv('Data/title_basics.csv.gz', compression='gzip', index=False)
ratings.to_csv('Data/title_ratings.csv.gz', compression='gzip', index=False)

In [25]:
# Open saved file and preview again
basics = pd.read_csv('Data/title_basics.csv.gz', low_memory=False)
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
